In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/PHD/SemEval2023/SemEval2022-Task10/private_space

/content/gdrive/MyDrive/PHD/SemEval2023/SemEval2022-Task10/private_space


In [3]:
# use github on google colab: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [160]:
!git config --global user.name "Pakawat Nakwijit"
!git config --global user.email "p.nakwijit@qmul.ac.uk"

In [162]:
ls

'Augmentation w  PMI.ipynb'     'Fine-tuning BERT TaskB C.ipynb'
'Augmentation w  XAI.ipynb'     'Lexicon Construciton [PMI].ipynb'
 Data/                          'Lexicon Construciton [Shapley].ipynb'
'Data Exploration.ipynb'        'Logistic Regression with BoW.ipynb'
 Evaluation.ipynb                Results/
'Fine-tuning BERT TaskA.ipynb'  'Run BERT.ipynb'


In [161]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [5]:
ls -al Results/lexicon_*

-rw------- 1 root root  66018 Jan 12 14:33 Results/lexicon_pmi.json
-rw------- 1 root root 280150 Jan 12 18:49 Results/lexicon_shapley_augmented_v1.json
-rw------- 1 root root  84083 Jan 13 11:14 Results/lexicon_shapley_augmented_v2_shap.json
-rw------- 1 root root  83986 Jan 13 11:13 Results/lexicon_shapley_augmented_v2_sqrt.json
-rw------- 1 root root  42420 Jan 12 14:17 Results/lexicon_shapley.json


In [6]:
import json

with open('Results/lexicon_pmi.json') as fin:
    lexicon_pmi = json.load(fin)

with open('Results/lexicon_shapley.json') as fin:
    lexicon_shapley = json.load(fin)

with open('Results/lexicon_shapley_augmented_v1.json') as fin:
    lexicon_shapley_augmented_v1 = json.load(fin)

with open('Results/lexicon_shapley_augmented_v2_sqrt.json') as fin:
    lexicon_shapley_augmented_v2_sqrt = json.load(fin)

with open('Results/lexicon_shapley_augmented_v2_shap.json') as fin:
    lexicon_shapley_augmented_v2_shap = json.load(fin)

In [7]:
# (lexicon_shapley_augmented_v1['1. threats, plans to harm and incitement']["sister"])

In [8]:
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [13]:
import pandas as pd

target_column = "label_category"
data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

label_values = [
    '1. threats, plans to harm and incitement',
    '2. derogation',
    '3. animosity',
    '4. prejudiced discussions',
]

In [14]:
label_map = {
    0: '1. threats, plans to harm and incitement',
    1: '2. derogation', 
    2: '3. animosity', 
    3: '4. prejudiced discussions',
    '1. threats, plans to harm and incitement':0,
    '2. derogation':1,
    '3. animosity':2,
    '4. prejudiced discussions':3,
}

In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

## Lexicon counting

In [16]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
import string

def preprocess(sent, **kwargs):
  sent = sent.lower()
  words = nltk.word_tokenize(sent)

  # Remove Stop words
  words = [w for w in words if w not in stoplist]

  # Remove Punctuation
  words = [w for w in words if w not in string.punctuation]
  
  # Lematization
  if "lemma" in kwargs and kwargs["lemma"]:
    words = [lemmatizer.lemmatize(w) for w in words]
    
  return words

In [18]:
from sklearn.metrics import f1_score

def predict(words, lexicons):
  cnt = {}
  for label in lexicons:
    cnt[label] = 0

  for w in words:
    for label in lexicons:
      if w in lexicons[label]:
        cnt[label] += lexicons[label][w]

  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]


def run_eval(data, lexicons, **kwargs):
  texts = data["text"].values
  sents = [preprocess(t, **kwargs) for t in texts]
  labels = data[target_column].values

  y_pred = []
  y_test = []
  non = 0
  for words, label in zip(sents, labels):
    pred = predict(words, lexicons)
    if pred is None:
      non += 1
      continue

    y_pred.append(pred)
    y_test.append(label)

  print("F1:", f1_score(y_test, y_pred, average='macro'))
  print("Skip:", non/len(sents))
  return 

In [19]:
print("Evaluate PMI Lexicons")
run_eval(test, lexicon_pmi, lemma=True)

Evaluate PMI Lexicons
F1: 0.24188623890648447
Skip: 0.45294117647058824


In [20]:
print("Evaluate Shapley Lexicons")
run_eval(test, lexicon_shapley, lemma=False)

Evaluate Shapley Lexicons
F1: 0.2647482751854599
Skip: 0.11029411764705882


In [21]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v1, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.2787668443532826
Skip: 0.10294117647058823


In [22]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v2_sqrt, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.270143909309982
Skip: 0.10735294117647058


In [23]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v2_shap, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.2648284140658099
Skip: 0.10735294117647058


## Logistic Regression with all words

In [139]:
from sklearn.feature_extraction.text import CountVectorizer

def logistic_preprocess(sent):
  words = preprocess(sent, lemma=True)
  return words

def dummy_word_tokenize(sent):
  return sent

vectorizer = CountVectorizer(
    preprocessor=logistic_preprocess,
    tokenizer=dummy_word_tokenize
)

In [140]:
X_train = vectorizer.fit_transform(train["text"].values)
X_test = vectorizer.transform(test["text"].values)

In [141]:
y_train = [label_map[l] for l in train[target_column].values]
y_test = [label_map[l] for l in test[target_column].values]

In [142]:
"#Vocab", len(vectorizer.get_feature_names_out())

('#Vocab', 7574)

In [143]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

model = LogisticRegression(random_state=42, penalty="l2")
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [144]:
y_predict = model.predict(X_test)

In [149]:
print("F1: %.3f" % metrics.f1_score(y_test, y_predict, average='macro'))
print("Acc: %.3f" % metrics.accuracy_score(y_test, y_predict))

p, r, f1, sup = metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")
print("P: %.3f" % p)
print("R: %.3f" % r)

F1: 0.445
Acc: 0.504
P: 0.526
R: 0.415


## Logistic Regression with selected lexicons

In [103]:
class Preprocessor:
  def __init__(self, vocabs):
    self.vocabs = vocabs
  
  def run(self, sent):
    words = preprocess(sent, lemma=True)

    # Remove words that are not in vocabs
    words = [w for w in words if w in self.vocabs]
    return words


def train_reg(data, lexicon, **kwargs):
  vocabs = set()
  for label in lexicon:
    vocabs.update(lexicon[label].keys())

  preprocessor = Preprocessor(vocabs)
  vectorizer = CountVectorizer(
      preprocessor=preprocessor.run,
      tokenizer=dummy_word_tokenize
  )

  X_train = vectorizer.fit_transform(data["text"].values)

  y_train = [label_map[l] for l in data[target_column].values]
  
  print("#Vocab", len(vectorizer.get_feature_names_out()))

  model = LogisticRegression(random_state=42, penalty="l2")
  model.fit(X_train, y_train)

  return model, vectorizer


In [151]:
def run_eval_feat(data, model, vectorizer, **kwargs):
  y_test = [label_map[l] for l in data[target_column].values]
  X_test = vectorizer.transform(data["text"].values)

  y_predict = model.predict(X_test)
  print("F1: %.3f" % metrics.f1_score(y_test, y_predict, average='macro'))
  print("Acc: %.3f" % metrics.accuracy_score(y_test, y_predict))

  p, r, f1, sup = metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")
  print("P: %.3f" % p)
  print("R: %.3f" % r)
  return 

In [152]:
# F1: 0.445
# Acc: 0.504
# P: 0.526
# R: 0.415


In [153]:
print("Evaluate PMI Lexicons")
model, vectorizer = train_reg(train, lexicon_pmi)
run_eval_feat(test, model, vectorizer)

Evaluate PMI Lexicons
#Vocab 4556
F1: 0.232
Acc: 0.471
P: 0.382
R: 0.265


In [154]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley)
run_eval_feat(test, model, vectorizer)


Evaluate Shapley Lexicons
#Vocab 1037
F1: 0.324
Acc: 0.521
P: 0.514
R: 0.325


In [155]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v1)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 1326
F1: 0.324
Acc: 0.522
P: 0.496
R: 0.325


In [156]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v2_shap)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 1260
F1: 0.330
Acc: 0.524
P: 0.508
R: 0.329


In [157]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v2_sqrt)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 1260
F1: 0.330
Acc: 0.524
P: 0.508
R: 0.329


In [158]:
(7574-1260)/7574

0.833641404805915

In [42]:
(0.445-0.330)/0.445

0.2584269662921348

In [51]:
# # Grid search cross validation
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# import numpy as np

# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2", None], 'solver' : ['liblinear', 'lbfgs']}

# logreg=LogisticRegression()
# logreg_cv=GridSearchCV(logreg,grid,cv=5)
# logreg_cv.fit(X_train,y_train)

# print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
# print("accuracy :",logreg_cv.best_score_)